In [24]:
import pandas as pd
import json
from collections import defaultdict
import re
import datetime, time

#### Con este bucle obtenemos los datos sobre la inicialización de todos los niveles jugados

In [25]:
def extraerTiempos(rawData):
    erLevel = re.compile(r'\blevel$\b')
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erIdLevel = re.compile(r'/')
    
    data = defaultdict(defaultdict)
    anomalias = []

    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erLevel.search(obj) : #Si el objeto de la acción es un nivel
            name = evento["actor"]["name"]
            levelCode = erIdLevel.split(evento["object"]["id"])[-1]
            timestamp = evento["timestamp"]
            if erInitialized.search(verb): #Si la acción es inicio o reinicio
                if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                    if levelCode in data[name]:
                        data[name][levelCode].append({"ini" : timestamp, "fin" : "", "stars" : ""})
                    else:
                        data[name][levelCode] = [{"ini" : timestamp, "fin" : "", "stars" : ""}]
                #else: #-> sería cuando un nivel es reiniciado, por ahora no vamos a tenerlo en cuenta
            elif erCompleted.search(verb):
                if evento["result"]["success"]: #Significa que el nivel se ha completado con éxito
                    if levelCode in data[name]:
                        data[name][levelCode][-1]["fin"] = timestamp
                        data[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
                    else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                        anomalias.append(evento)
                else:
                    if levelCode in data[name]:
                        data[name][levelCode][-1]["fin"] = None #Porque el jugador sale al menú o cierra el juego
                    else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                        anomalias.append(evento)
    return data, anomalias              

In [26]:
#Esta clase sirve para parsear timestamps, recibe 2 y hace la diferencia
class Tiempo:
    def __init__ (self, inicio, final, seconds = -1): #Necesario para tener 2 constructores
        if seconds == -1: #Primer constructor (2 aregumentos)
            self.hours = 0
            self.minutes = 0
            self.seconds = 0

            fin = time.mktime(datetime.datetime.strptime(final, "%Y-%m-%dT%H:%M:%S.%fZ").timetuple())
            ini = time.mktime(datetime.datetime.strptime(inicio, "%Y-%m-%dT%H:%M:%S.%fZ").timetuple())

            hoursRaw = int((fin - ini)/3600)
            minutesRaw = int((fin - ini)/60)
            secondsRaw = int(fin - ini)

            if hoursRaw != 0:
                self.hours = hoursRaw
                minutesRaw = int(((fin - ini)%3600)/60)
            if minutesRaw != 0:
                self.minutes = minutesRaw
                secondsRaw = int((fin - ini)%60)
            self.seconds = secondsRaw
            
        else: #Segundo constructor (3 argumentos)
            self.hours = hours
            self.minutes = minutes
            self.seconds = seconds

    def toString(self):
        t=""
        if self.hours !=0:
            t += str(self.hours) + "h/"
        if self.minutes !=0:
            t += str(self.minutes) + "m/"
        if self.seconds !=0:
            t += str(self.seconds) + "s"
        return t
    
    def __add__(self, t): #Sobrecarga del operador +
        s = self.seconds + t.seconds
        m = self.minutes + t.minutes
        h = self.hours + t.hours
        
        if s >= 60:
            s -= 60
            m += 1
        if m >= 60:
            m -= 60
            h += 1
        return Tiempo(h,m,s)

In [27]:
def tiempoPorNiveles(data, anomalias):
    tiemposJugados = defaultdict(defaultdict)
    for player in data:
        for level in data[player]:
            for times in data[player][level]:
                if times["fin"] != None: #Si no se aborto el intento del nivel
                    try:
                        timeDifference = Tiempo(times["ini"], times["fin"]).toString()
                        if level in tiemposJugados[player]:
                            tiemposJugados[player][level].append({"time" : timeDifference, "stars" : times["stars"]})
                        else:
                            tiemposJugados[player][level] = [{"time" : timeDifference, "stars" : times["stars"]}]
                    except ValueError: #Algunos timestamps de finalizacion estan vacios, ¿Tiene que ser asi?
                        anomalias.append({"jugador" : player,
                                  "nivel" : level,
                                  "Timestamp-Inicio" : times["ini"],
                                  "Timestamp-Fin" : times["fin"],
                                  "Descripcion:" : "No se ha podido parsear uno de los 2 timestamps"})
    return tiemposJugados

In [28]:
def buscarTarjetas(rawData):
    data = []
    erIdLevel = re.compile(r'/')
    for evento in rawData:
        idJugador = evento["actor"]["name"]
        levelCode = erIdLevel.split(evento["object"]["id"])[-1]
        if idJugador == "bdwyj" and "result" in evento and "extensions" in evento["result"] and "level" in evento["result"]["extensions"]:
            if evento["result"]["extensions"]["level"] == "variables_9":
                data.append(evento)
        elif idJugador == "bdwyj" and levelCode == "variables_9":
            data.append(evento)
    return data

In [31]:
#ESTA MAL PORQUE LO SUMA COMO STRING EN VEZ DE COMO OBJETO "TIEMPO"
def getMediaTiemposYEstrellas(data):
    tiemposYEstrellasMedios = defaultdict()
    for user in data:
        for level in data[user]:
            for time in data[user][level]:
                if level in tiemposYEstrellasMedios:
                    tiemposYEstrellasMedios[level]["tiempo"] += time["time"]
                    tiemposYEstrellasMedios[level]["stars"] += time["stars"]
                else:
                    tiemposYEstrellasMedios[level] = {"tiempo" : time["time"], "stars" : time["stars"]}
    return tiemposYEstrellasMedios

In [32]:
JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)
data, anomalias = extraerTiempos(rawData)
JSONFile.close()

pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None
#pd.DataFrame(data)
#dataTarjetas = buscarTarjetas(rawData)
#pd.DataFrame(dataTarjetas)
tiempos = tiempoPorNiveles(data, anomalias)
#pd.DataFrame(tiempos)
tiemposMedios = getMediaTiemposYEstrellas(tiempos)
pd.DataFrame(tiemposMedios)

,tutorials_1,tutorials_2,tutorials_3,tutorials_4,tutorials_5,tutorials_6,tutorials_7,variables_1,variables_2,variables_3,variables_4,variables_5,variables_6,variables_7,variables_8,variables_9,types_1,types_2,types_3,types_4,types_5,types_6,types_7,types_8,types_9,basic_operators_1,basic_operators_2,basic_operators_3,basic_operators_4,basic_operators_5,basic_operators_6,basic_operators_7,loops_1,loops_2,loops_3
tiempo,19s48s32s11s40s25s25s57s40s52s26s25s41s1m/6s17s19s42s1m/35s40s42s28s1m/22s33s39s29s23s26s33s1m/3s1m/11s1m/14s29s20s1m/27s31s22s1m/8s27s55s17s29s25s1m/5s37s21s25s26s1m/12s34s1m/11s24s1m/3s1m/32s36s56s29s1m/16s39s37s32s31s2m/34s1m/3s47s27s36s1m/4s1m/20s2m/24s35s37s25s1m/8s1m/17s1m/11s1m/32s21s41s32s14s21s14s35s1m/13s49s32s33s32s2m/34s18s51s1m/29s1m/1s2m/1s21s1m/28s34s28s46s1m/19s1m/10s39s1m/49s1m/17s1m/10s30s1m/1m/1s24s2m/8s37s1m/21s2m/17s11s59s54s1m/32s2m/20s1m/19s1m/59s1m/2s1m/11s1m/11s1m/42s24s36s58s16s51s4m/25s1m/30s1m/26s1m/36s2m/49s1m/26s1m/25s2m/24s1m/1m/31s2m/35s1m/17s2m/46s1m/46s1m/34s1m/35s20s1m/40s,25s51s28s1m/12s39s26s21s51s23s50s28s33s31s26s28s25s38s30s49s1m/1s40s22s30s24s24s32s27s22s32s33s57s29s32s39s23s50s28s27s27s1m/4s33s27s21s24s30s26s28s59s29s20s1m/27s31s58s33s35s4m/37s1m/1s1m/14s39s28s1m/45s59s20s34s45s41s1m/13s44s8m/22s46s1m/21s1m/27s1m/35s21s22s33s32s43s39s25s30s30s28s25s26s29s48s51s23s1m/48s1m/25s28s3m/5s27s42s2m/14s29s1m/21s1m/11s30s26s40s21s3m/32s1m/37s46s47s21s32s2m/34s35s32s23s51s1m/16s34s3m/19s55s1m/11s3m/44s22s37s31s23s30s24s28s1m/7s15s26s45s1m/21s23s56s2m/21s56s26s1m/12s44s26s2m/5s40s1m/27s37s1m/29s16s56s30s33s27s2m/18s,20s1m/3s22s23s22s35s19s29s19s20s37s42s28s17s22s37s44s31s29s22s35s24s28s22s18s25s51s56s23s31s34s27s22s27s24s44s24s49s34s24s22s21s24s1m/15s1m/2s39s21s1m/29s23s28s34s20s27s44s35s22s23s17s48s27s30s19s21s30s35s33s29s37s30s37s56s33s29s1m/9s25s26s27s21s33s18s14s33s22s21s25s20s33s23s54s34s37s24s21s57s24s27s28s1m/4s51s32s51s29s18s23s35s30s26s23s25s27s2m/2s21s1m/9s33s29s44s53s32s30s40s31s1m/21s1m/49s33s36s20s48s22s28s31s1m/5s30s21s22s42s2m/42s1m/11s1m/15s29s1m/3s19s1m/13s38s41s1m/39s24s31s40s3m/23s30s,1m/7s4m/51s2m/33s2m/25s2m/2s1m/30s3m/7s1m/34s1m/42s2m/9s2m/42s1m/19s1m/28s1m/30s2m/21s1m/48s3m/25s2m/13s2m/1s1m/30s1m/28s1m/20s1m/51s1m/31s2m/24s1m/56s1m/29s3m/46s2m/55s2m/47s2m/22s1m/29s1m/52s1m/52s1m/42s2m/6s2m/18s1m/30s1m/24s1m/30s2m/43s2m/6s2m/30s2m/23s1m/52s2m/19s5m/24s2m/22s4m/7s6m/28s4m/31s5m/5m/25s1m/25s2m/54s4m/24s3m/47s3m/10s4m/34s2m/2s1m/24s1m/7s3m/33s3m/58s2m/35s5m/49s2m/32s15m/7s2m/3s1m/46s2m/14s3m/25s3m/4s2m/8s1m/51s2m/36s1m/30s3m/23s5m/57s1m/9s1m/53s1m/24s2m/1s2m/3s2m/44s2m/43s2m/56s1m/20s7m/51s2m/33s2m/8s54s2m/55s1m/52s2m/33s2m/20s1m/52s2m/53s2m/29s1m/22s1m/18s1m/19s1m/48s1m/32s1m/59s2m/1s5m/41s1m/42s7m/39s4m/51s2m/36s4m/6s1m/33s5m/17s3m/57s2m/10s3m/44s1m/22s2m/21s3m/17s2m/58s2m/51s8m/28s2m/3s8m/49s6m/11s1m/30s5m/56s43s11m/37s3m/57s2m/43s5m/7s4m/21s2m/22s3m/5s1m/55s2m/38s3m/22s2m/31s4m/36s2m/47s3m/25s1m/13s1m/38s1m/24s11m/7m/54s4m/33s,25s26s24s19s2m/55s19s50s15s39s27s31s18s27s19s16s41s17s16s36s40s18s23s15s13s20s24s34s16s29s27s13s23s23s30s42s35s32s27s1m/29s25s55s17s46s55s55s33s18s37s17s19s20s33s28s20s27s35s22s23s24s32s24s27s31s17s19s30s33s22s20s1m/13s1m/25s29s38s15s1m/7s21s30s15s12s21s14s35s17s12s31s22s19s41s25s37s24s1m/3s28s29s27s27s26s14s24s28s23s1m/13s47s19s37s23s22s1m/10s1m/1s48s44s35s40s20s1m/9s37s25s37s31s27s43s24s29s1m/57s26s26s17s25s27s1m/20s34s41s30s25s34s41s54s46s27s59s23s18s32s33s22s29s,1m/4s1m/33s1m/32s1m/15s2m/58s1m/6s1m/47s1m/33s1m/10s1m/40s1m/39s1m/37s1m/10s1m/40s1m/53s1m/49s1m/29s1m/55s2m/9s3m/4s1m/15s1m/38s1m/37s1m/10s1m/27s2m/26s1m/38s2m/9s1m/34s2m/15s1m/58s1m/7s1m/47s1m/24s1m/19s1m/22s2m/13s1m/6s1m/18s1m/13s1m/18s2m/39s2m/10s1m/58s1m/29s1m/43s2m/2m/1s2m/10s2m/35s3m/38s1m/54s2m/16s2m/6s2m/16s3m/41s2m/33s1m/24s1m/51s2m/43s1m/57s1m/7s1m/17s1m/22s1m/42s2m/46s1m/30s2m/19s3m/19s2m/25s1m/31s2m/15s2m/42s1m/37s1m/29s1m/57s2m/6s1m/37s2m/15s1m/35s1m/2s1m/54s59s2m/53s1m/42s1m/14s2m/5s2m/15s1m/21s5m/11s1m/56s56s1m/7s45s1m/24s1m/32s2m/18s2m/43s2m/16s3m/14s1m/49s1m